In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio as psnr
import pandas as pd
from IPython.display import display, HTML
import base64
from scipy.signal import convolve2d

# Configuration
INPUT_ROOT = 'DATASET'
OUTPUT_ROOT = 'DATASET_wiener'
SSIM_THRESHOLD = 0.85
MAX_PREVIEW = 20

# Wiener Filter function
def wiener_filter(image, kernel_size=5):
    """Apply Wiener filter to an image."""
    kernel = np.ones((kernel_size, kernel_size)) / kernel_size**2
    image_blur = convolve2d(image, kernel, mode="same", boundary="wrap")
    noise = image - image_blur
    denoised_image = image_blur + noise
    return denoised_image

def calculate_ssim(original, denoised):
    """Calculate SSIM between the original and denoised images."""
    return structural_similarity(original, denoised, data_range=255)

def calculate_psnr(original, denoised, data_range=255):
    """Calculate PSNR between the original and denoised images."""
    mse = np.mean((original - denoised) ** 2)
    if mse == 0:  # Identical images
        return float('inf')  # Return infinity to indicate perfect similarity
    return 10 * np.log10((data_range ** 2) / mse)

def encode_image_for_html(img):
    """Encode image into base64 for embedding into HTML."""
    _, buffer = cv2.imencode('.jpg', img)
    b64 = base64.b64encode(buffer).decode()
    return f'<img src="data:image/jpeg;base64,{b64}" width="100"/>'

def process_dataset_wiener():
    """Process the dataset with Wiener filtering and compute SSIM and PSNR."""
    report = []
    total_ssim = 0
    total_psnr = 0
    processed_count = 0

    total_images = sum(
        len(files)
        for _, _, files in os.walk(INPUT_ROOT)
        if any(f.lower().endswith(('.png', '.jpg', '.jpeg')) for f in files)
    )

    progress = tqdm(total=total_images, desc="🔬 Wiener Filtering", unit="img")

    for split in os.listdir(INPUT_ROOT):
        split_path = os.path.join(INPUT_ROOT, split)
        if not os.path.isdir(split_path):
            continue

        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if not os.path.isdir(class_path):
                continue

            for filename in os.listdir(class_path):
                if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    continue

                img_path = os.path.join(class_path, filename)
                image = cv2.imread(img_path)
                if image is None:
                    continue

                # Split the image into its BGR channels
                b, g, r = cv2.split(image)

                # Apply Wiener filter on each channel individually
                b_denoised = wiener_filter(b)
                g_denoised = wiener_filter(g)
                r_denoised = wiener_filter(r)

                # Clip the values to make sure they're within the proper range (0-255)
                b_denoised = np.clip(b_denoised, 0, 255).astype(np.uint8)
                g_denoised = np.clip(g_denoised, 0, 255).astype(np.uint8)
                r_denoised = np.clip(r_denoised, 0, 255).astype(np.uint8)

                # Merge the denoised channels back together
                denoised_image = cv2.merge([b_denoised, g_denoised, r_denoised])

                # Calculate SSIM
                final_ssim = calculate_ssim(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY))

                # Calculate PSNR
                final_psnr = calculate_psnr(image, denoised_image)

                # If PSNR is infinite (identical images), handle it gracefully
                if np.isinf(final_psnr):
                    final_psnr = float('nan')  # Or any value you wish to represent identical images

                # Save output
                output_path = os.path.join(OUTPUT_ROOT, split, class_name, filename)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                cv2.imwrite(output_path, denoised_image)

                # For preview table
                if processed_count < MAX_PREVIEW:
                    preview = encode_image_for_html(denoised_image)
                else:
                    preview = ""

                report.append({
                    'Image': preview,
                    'Path': img_path,
                    'SSIM': f"{final_ssim:.4f}",
                    'PSNR': f"{final_psnr:.2f} dB" if not np.isnan(final_psnr) else "Identical Image"
                })

                total_ssim += final_ssim
                total_psnr += final_psnr if not np.isnan(final_psnr) else 0  # Skip if PSNR is "NaN"
                processed_count += 1
                progress.set_postfix({"Processed": processed_count, "Avg SSIM": total_ssim / processed_count if processed_count else 0})
                progress.update(1)

    progress.close()

    # Calculate average SSIM and PSNR
    avg_ssim = total_ssim / processed_count if processed_count > 0 else 0
    avg_psnr = total_psnr / processed_count if processed_count > 0 else 0
    return report, avg_ssim, avg_psnr

def show_html_table(report, avg_ssim, avg_psnr):
    """Display a table in HTML format with image previews, SSIM, and PSNR."""
    df = pd.DataFrame(report[:MAX_PREVIEW])
    print(f"\n✅ Average SSIM: {avg_ssim:.2%} across {len(report)} images.")
    print(f"✅ Average PSNR: {avg_psnr:.2f} dB across {len(report)} images.")
    display(HTML(df.to_html(escape=False, index=False)))

if __name__ == "__main__":
    report, avg_ssim, avg_psnr = process_dataset_wiener()
    show_html_table(report, avg_ssim, avg_psnr)


🔬 Wiener Filtering:   0%|          | 13/2853 [00:04<17:20,  2.73img/s, Processed=13, Avg SSIM=1]

: 